In [1]:
import pandas as pd
import numpy as np
import sys, os
import time
import io
import re

samPath = '/Users/jacky/Documents/py3samsdk-master'
if samPath not in sys.path:
    sys.path.insert(0, samPath)
    

from py3samsdk.sscapi import PySSC
from urllib.error import HTTPError
from urllib.request import urlopen

ssc_lib = '/Applications/sam-sdk-2017-9-5/osx64/'  # path to SAM SSC Library
ssc = PySSC(ssc_lib)

In [2]:
# Reads input file for email and api key
inputfile = open(os.getcwd() + "/input.txt", "r")
lines = inputfile.readlines()

In [3]:
your_email = re.search(r'[\w.]+@[\w]+.[\w]+', lines[0]).group()
api_key = re.search(r'[\w]+$', lines[1]).group()

In [4]:
counties = pd.read_csv("project-sunroof-county-08272017.csv").loc[:,['region_name', 'state_name',  'lat_avg', 'lng_avg', 'number_of_panels_f', 'number_of_panels_total', 'yearly_sunlight_kwh_f']]
counties['capacity'] = counties['number_of_panels_f']*0.00025

In [5]:
latlon = counties
latlon.head()

,region_name,state_name,lat_avg,lng_avg,number_of_panels_f,number_of_panels_total,yearly_sunlight_kwh_f,capacity
0,NaN,NaN,32.551544,-116.920065,0,31.0,0.000000e+00,0.00000
1,Acadia Parish,Louisiana,30.256188,-92.331117,10620,11642.0,3.557943e+06,2.65500
2,Ada County,Idaho,43.614095,-116.308027,2865925,7931427.0,9.644616e+08,716.48125
3,Adams County,Colorado,39.879721,-104.923252,3252408,6207759.0,1.160027e+09,813.10200
4,Aiken County,South Carolina,33.517511,-81.801685,834015,2840728.0,2.772775e+08,208.50375


In [6]:
year = 2011
interval = '60'
# leap_year = 'true'
yearly_generations = np.zeros(len(latlon))
hourly_generations = pd.DataFrame()
errors = np.zeros(len(latlon))
for index, row in latlon.iterrows():
    lat = row['lat_avg']
    lon = row['lng_avg']
    capacity = row['capacity']
    
    url = 'http://developer.nrel.gov/api/solar/nsrdb_0512_download.csv?wkt=POINT({lon}%20{lat})&names={year}&interval={interval}&email={email}&api_key={api}'.format(year=year, lat=lat, lon=lon, interval=interval, email=your_email, api=api_key)
    
    data = pd.DataFrame()
    trys = 1
    succeeded = False
    errored = False
    while(not succeeded):
        try:
            succeeded = True
            response = urlopen(url)
            data = pd.read_csv(io.StringIO(response.read().decode('utf-8')))
            time.sleep(5.5)
            trys += 1
        except HTTPError as error:
            succeeded = False
            if trys > 4:
                raise
            if error.code == 504:
                print('Attempting try number {0}'.format(trys))
            else:
                print('{0}. Error {1} with {2},{3}'.format(index, error.code, row['region_name'], row['state_name']))
                yearly_generations[index] = None
                errored = True
                errors[index] = error.code
                break
        except:
            latlon['generations'] = yearly_generations
            hourly_generations.to_csv('hourly' + str(year) + '.csv')
            latlon.to_csv('year' + str(year) + '.csv')
            raise
    
    if errored:
        continue
    else:
        errors[index] = None
    
    headers = dict(response.getheaders())
    remaining_calls = headers['X-RateLimit-Remaining']
    

    metadata = data.iloc[0:1, :]
    
    timezone = metadata['Time Zone'].values[0]
    elevation = metadata['Elevation'].values[0]
    
    loc_data = data.iloc[:,:]
    loc_data.columns = loc_data.iloc[1]
    loc_data = loc_data.iloc[2:, :]
    loc_data = loc_data.set_index(pd.date_range('1/1/{yr}'.format(yr=year), freq=interval+'Min', periods=525600/int(interval)))
    loc_data = loc_data.dropna(axis = 1, how='all')
    loc_data[['DNI','DHI', 'Wind Speed', 'Temperature']] = loc_data[['DNI','DHI', 'Wind Speed', 'Temperature']].apply(pd.to_numeric)
    
    wfd = ssc.data_create()
    ssc.data_set_number(wfd, 'lat', lat)
    ssc.data_set_number(wfd, 'lon', lon)
    ssc.data_set_number(wfd, 'tz', float(timezone))
    ssc.data_set_number(wfd, 'elev', float(elevation))
    ssc.data_set_array(wfd, 'year', loc_data.index.year)
    ssc.data_set_array(wfd, 'month', loc_data.index.month)
    ssc.data_set_array(wfd, 'day', loc_data.index.day)
    ssc.data_set_array(wfd, 'hour', loc_data.index.hour)
    ssc.data_set_array(wfd, 'minute', loc_data.index.minute)
    ssc.data_set_array(wfd, 'dn', loc_data['DNI'])
    ssc.data_set_array(wfd, 'df', loc_data['DHI'])
    ssc.data_set_array(wfd, 'wspd', loc_data['Wind Speed'])
    ssc.data_set_array(wfd, 'tdry', loc_data['Temperature'])

    # Create SAM compliant object  
    dat = ssc.data_create()
    ssc.data_set_table(dat, 'solar_resource_data', wfd)
    ssc.data_free(wfd)

    # Specify the system Configuration
    # Set system capacity in MW
    system_capacity = capacity
    ssc.data_set_number(dat, 'system_capacity', system_capacity)
    # Set DC/AC ratio (or power ratio). See https://sam.nrel.gov/sites/default/files/content/virtual_conf_july_2013/07-sam-virtual-conference-2013-woodcock.pdf
    ssc.data_set_number(dat, 'dc_ac_ratio', 1.15)
    # Set tilt of system in degrees
    # For Google data, roof segments are considered Flat for roofs with a tilt of less than 10%
    ssc.data_set_number(dat, 'tilt',0)
    # Set azimuth angle (in degrees) from north (0 degrees)
    ssc.data_set_number(dat, 'azimuth', 180)
    # Set the inverter efficency
    ssc.data_set_number(dat, 'inv_eff', 96)
    # Set the system losses, in percent
    ssc.data_set_number(dat, 'losses', 14.0757)
    # Specify fixed tilt system (0=Fixed, 1=Fixed Roof, 2=1 Axis Tracker, 3=Backtracted, 4=2 Axis Tracker)
    ssc.data_set_number(dat, 'array_type', 0)
    # Set ground coverage ratio
    ssc.data_set_number(dat, 'gcr', 1)
    # Set constant loss adjustment
    ssc.data_set_number(dat, 'adjust:constant', 0)

    # execute and put generation results back into dataframe
    mod = ssc.module_create('pvwattsv5')
    ssc.module_exec(mod, dat)
    loc_data['generation'] = np.array(ssc.data_get_array(dat, 'gen'))
    hourly_generations[row['region_name']] = loc_data['generation']
    
    # free the memory
    ssc.data_free(dat)
    ssc.module_free(mod)
    print('{0}. {1}: {2}; Remaining Calls: {3}'.format(index, row['region_name'], loc_data['generation'].sum(), remaining_calls))
    yearly_generations[index] = loc_data['generation'].sum()*1000
latlon['generations'] = yearly_generations
latlon['error codes'] = errors

0. nan: 0.0; Remaining Calls: 1754
1. Acadia Parish: 3627.104525477276; Remaining Calls: 1753
2. Ada County: 888194.8739270568; Remaining Calls: 1752
3. Adams County: 1091412.7418691516; Remaining Calls: 1751
4. Aiken County: 275365.68570875004; Remaining Calls: 1750
5. Alachua County: 579177.2143434286; Remaining Calls: 1749
6. Alamance County: 271795.9562628139; Remaining Calls: 1748
7. Alameda County: 3978915.840152517; Remaining Calls: 1747
8. Albany County: 760930.88649185; Remaining Calls: 1746
9. Albemarle County: 107095.66032395558; Remaining Calls: 1745
10. Alexander County: 8306.722483609978; Remaining Calls: 1744
11. Alexander County: 749.6516140913482; Remaining Calls: 1743
12. Allegan County: 127118.62783764768; Remaining Calls: 1742
13. Allegany County: 63942.9623272231; Remaining Calls: 1741
14. Allegheny County: 1624249.8594369963; Remaining Calls: 1740
15. Allen County: 119859.11098652307; Remaining Calls: 1739
16. Allen County: 825118.1791084412; Remaining Calls: 1738

ValueError: Length mismatch: Expected axis has 8173 elements, new values have 8760 elements

In [ ]:
hourly_generations.head()

In [ ]:
latlon.head()

In [ ]:
hourly_generations.to_csv('hourly' + str(year) + '.csv')
latlon.to_csv('year' + str(year) + '.csv')

In [ ]:
latlon = pd.read_csv('yearly.csv')

In [ ]:
latlon['percent ratio'] = latlon['generations'] / latlon['yearly_sunlight_kwh_f']

In [ ]:
%matplotlib inline
latlon['percent ratio'].plot(kind='hist')

In [ ]:
from matplotlib import pyplot as plt
plt.style.use('ggplot')

def nsrdb_plot(df, i):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax2 = ax.twinx()
    df['90 Degree Zenith'] = 90
    df[['GHI', 'DNI', 'DHI', 'Solar Zenith Angle', '90 Degree Zenith']][i:i+int(interval)].plot(ax=ax, figsize=(15,8), yticks=(np.arange(0,900,100)), style={'90 Degree Zenith': '--','Solar Zenith Angle': '-o', 'DNI': '-o', 'DHI': '-o', 'GHI': '-o'}, legend=False)
    df['generation'][i:i+30].plot(ax=ax2, yticks=(np.arange(0,4.5,0.5)), style={'generation': 'y-o'})
    ax.grid()
    ax.set_ylabel('W/m2')
    ax2.set_ylabel('kW')
    ax.legend(loc=2, ncol=5, frameon=False)
    ax2.legend(loc=1, frameon=False)

In [ ]:
nsrdb_plot(df, 5050)